In [1]:
%load_ext lab_black

Step: Load data

In [2]:
import pandas as pd
import os
import numpy as np
import nltk
from pyprojroot import here

from skimpy import clean_columns
from data_cleaning.fun_hot_encode_limit import fun_hot_encode_limit

path_data = here("./data")
os.chdir(path_data)
data_ra = pd.read_csv("ra_data.csv")
data_ra = clean_columns(data_ra)

nltk.download("vader_lexicon")
data_ra["date"] = pd.to_datetime(data_ra["date"])

11 column names have been cleaned

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
/tmp/ipykernel_27899/3187627048.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_ra["date"] = pd.to_datetime(data_ra["date"])


In [4]:
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()

# Let's assume your DataFrame is called df and the column with the text data is 'reviews'
data_ra["sentiment_scores"] = data_ra["reviews"].apply(
    lambda review: sia.polarity_scores(review)
)

# The above will return a dictionary with 'compound', 'neg', 'neu', 'pos' as keys.
# If you're interested in overall sentiment, you could use 'compound' score which is computed by summing the valence scores of each word in the lexicon, adjusted according to the rules, and then normalized to be between -1 (most extreme negative) and +1 (most extreme positive).
# You can create a new column for this score.

data_ra["compound_score"] = data_ra["sentiment_scores"].apply(
    lambda score_dict: score_dict["compound"]
)

In [5]:
data_ra_sentiment = data_ra.copy()

In [6]:
data_ra_sentiment["compound_score"].describe()

count    185.000000
mean      -0.013512
std        0.609303
min       -0.980300
25%       -0.539900
50%        0.000000
75%        0.542300
max        0.991500
Name: compound_score, dtype: float64

In [7]:
data_ra_sentiment_high = data_ra_sentiment.query("(compound_score >= .5)").reset_index(
    drop=True
)

data_ra_sentiment_low = data_ra_sentiment.query("(compound_score <= -.5)").reset_index(
    drop=True
)

In [8]:
# Import necessary libraries
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

# Extract reviews
reviews_high = data_ra_sentiment_high['reviews'].values

# Create BERTopic model
topic_model_high = BERTopic(language="english")

# Fit the model to the reviews
topics, _ = topic_model_high.fit_transform(reviews_high)

# Get the topics
topic_model_high.get_topic_freq().head()


ImportError: Numba needs NumPy 1.22 or less